In [14]:
import pandas as pd

## Data Preparation

In [15]:
#load data
df_seq=pd.read_csv('pdb_data_seq.csv')
df_properties=pd.read_csv('pdb_data_no_dups.csv')
df_total=df_seq.merge(df_properties,left_on='structureId',right_on = 'structureId')


In [16]:
df_total.columns

Index(['structureId', 'chainId', 'sequence', 'residueCount_x',
       'macromoleculeType_x', 'classification', 'experimentalTechnique',
       'macromoleculeType_y', 'residueCount_y', 'resolution',
       'structureMolecularWeight', 'crystallizationMethod',
       'crystallizationTempK', 'densityMatthews', 'densityPercentSol',
       'pdbxDetails', 'phValue', 'publicationYear'],
      dtype='object')

#### Select only protein, and filtered by top N

In [17]:
#select the data in top n by count, top 10?
df_total=df_total[df_total['macromoleculeType_x'].isin(set(['Protein']))]

count = df_total['classification'].value_counts(dropna=False)[:10]
df_selected=df_total[df_total['classification'].isin(set(count.index))]
#we want only protein
#count = df_selected['classification'].value_counts(dropna=True)[:10]

In [18]:
count

HYDROLASE                        46336
TRANSFERASE                      36424
OXIDOREDUCTASE                   34322
IMMUNE SYSTEM                    15615
LYASE                            11682
HYDROLASE/HYDROLASE INHIBITOR    11218
TRANSCRIPTION                     8919
VIRAL PROTEIN                     8495
TRANSPORT PROTEIN                 8371
VIRUS                             6972
Name: classification, dtype: int64

#### Select proteins with only one chain

In [19]:
#select proteins with only one chain in the data set
#how to justify this operation?
#df_onechain = df_selected[df_selected.groupby('structureId').structureId.transform(len) == 1]

In [20]:
test_df = df_selected[['structureId','classification','sequence']]
#test_df = df_onechain[['structureId','classification','sequence']]

Things to be done

Further select data and simplify problem, select proteins with only one chain?


Figure out how to convert sequence data into array and training model afterwards. (22 - n -n)


More models and discussion (*LSTM)

Models build on features other than sequence.



## Data balancing

In [21]:
from sklearn.utils import resample
#upsample the minorities and then take 50000 sample from all

df_majority = test_df[test_df['classification']=='HYDROLASE']
df_minority = test_df[test_df['classification']=='HYDROLASE']
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=46336,    # to match majority class
                                 random_state=123) # reproducible results
keylist = list(count.index)
datalist = [0]*9
for i in range(1,10):
    df_minority = test_df[test_df['classification']==keylist[i]]
    datalist[i-1] = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=46336,    # to match majority class
                                 random_state=123) # reproducible results
    #print(len(df_majority))
    df_majority = pd.concat([df_majority, datalist[i-1]])
data = df_majority.sample(50000)
    

## Model Training

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier

In [23]:
#optional, take part of the data for faster verification
#data = test_df#.sample(50000)

#need to remove nulls
data = data.dropna()
X_train, X_test,y_train,y_test = \
train_test_split(data['sequence'], data['classification'], test_size = 0.1, random_state = 1)

In [24]:
test_count = data['classification'].value_counts(dropna=False)[:10]

In [25]:
np.asarray(test_count.index)

NameError: name 'np' is not defined

In [26]:
import numpy as np

#### Feature Extraction From Sequence Data

In [ ]:
"""
#vectorize data, prepare for building models
#Convert a collection of text documents to a matrix of token counts
#seems has nothing to do with sequence but only with the frequency

#ngram is a parameter we need to focus on, 

vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (1,1))
#vect = CountVectorizer(analyzer = 'char_wb')

# Fit and Transform CountVectorizer
#occasionally may meet np.nan error
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)

#to store the results for different mothods
prediction = dict()
"""

Iterate through models

In [27]:
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,recall_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.neural_network import MLPClassifier
import time

In [28]:
models = [MultinomialNB(),AdaBoostClassifier(),RandomForestClassifier(),KNeighborsClassifier(),DecisionTreeClassifier(),ExtraTreesClassifier(),SVC()]

model_names=['NB','adaboost','randomForest','Kneighbors','DecisionTree','ExtraTree','SVM']
extra_models = dict(zip(model_names,models))

In [29]:
count

HYDROLASE                        46336
TRANSFERASE                      36424
OXIDOREDUCTASE                   34322
IMMUNE SYSTEM                    15615
LYASE                            11682
HYDROLASE/HYDROLASE INHIBITOR    11218
TRANSCRIPTION                     8919
VIRAL PROTEIN                     8495
TRANSPORT PROTEIN                 8371
VIRUS                             6972
Name: classification, dtype: int64

In [ ]:
precisions={}
recalls={}
confusion_matrixs={}
types  = np.asarray(test_count.index)
score_table = {}
from tqdm import tqdm
init_time = time.time()
i=5
#featrue extraction by count vectorization
vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (i,i))
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)
for key in extra_models.keys():
    model = extra_models[key]
    model.fit(X_train_df,y_train)
    ADA_pred = model.predict(X_test_df)
    if key not in precisions:
        precisions[key] = [accuracy_score(ADA_pred , y_test)]
        #recalls[key] = [recall_score(ADA_pred , y_test)]
        confusion_matrixs[key] = [confusion_matrix(ADA_pred, y_test, labels = types)]
        score_table[key] = [classification_report(ADA_pred,y_test, labels = types)]
    else:
        precisions[key].append(accuracy_score(ADA_pred , y_test))  
        #recalls[key].append(recall_score(ADA_pred, y_test))
        confusion_matrixs[key].append(confusion_matrix(ADA_pred,y_test, labels = types))
        score_table[key].append( classification_report(ADA_pred,y_test, labels = types))
    print('time elapsed:',time.time()-init_time)
    print(i,key,accuracy_score(ADA_pred , y_test))

time elapsed: 36.29348945617676
5 NB 0.8856


C:\Users\Weinan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


time elapsed: 111.03484630584717
5 adaboost 0.353


C:\Users\Weinan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


time elapsed: 990.1190183162689
5 randomForest 0.8832
time elapsed: 999.3678994178772
5 Kneighbors 0.7512
